In [46]:
import numpy as np
from numba import njit
import time

In [29]:
mat1 = np.array(
    [
        [0.3, 0.4, 1.2],
        [0.7, 0.1, 0.1],
        [1.7, 0.6, 0.1],
    ]

)
mat1

array([[0.3, 0.4, 1.2],
       [0.7, 0.1, 0.1],
       [1.7, 0.6, 0.1]])

In [75]:
@njit(cache=True)
def bucket_rounding(mat):
    if mat.ndim != 2:
        raise ValueError("Input must be a 2-dimensional numpy array")

    rounded = np.zeros_like(mat, dtype=np.double)

    n = mat.shape[0]
    for i in range(n):
        residual = 0
        for j in range(n):
            if mat[i, j] != 0:
                val = round(mat[i, j] + residual)
                residual += mat[i, j] - val
                rounded[i, j] = val

    total_diff = np.round(rounded.sum() - mat.sum())
    if total_diff > 0:
        diff = -1
    else:
        diff = 1
    indices = np.argsort(np.diag(rounded))[::-1].astype(np.int16)[:np.abs(total_diff)]
    for i in range(indices.shape[0]):
        rounded[int(indices[i]), int(indices[i])] += diff
    
    for i in range(rounded.shape[0]):
        for j in range(rounded.shape[1]):
            rounded[i, j] = max(0, rounded[i, j])
    return rounded.astype(np.int32)

In [76]:
def test_bucket_rounding(size, input_matrix=None):
    if input_matrix is None:
        mat = np.random.random((size,size))
    else:
        mat = input_matrix
    print("Test Matrix Shape = {:} * {:}".format(*mat.shape))
    
    start_time = time.time()
    mat_rounded = bucket_rounding(mat)
    end_time = time.time()
    execution_time = end_time-start_time
    print("Execution completed in %.2f seconds" % execution_time)
    
    row_diff = np.max(np.abs(mat_rounded.sum(axis=0) - mat.sum(axis=0)))
    print("Maximum row difference = {:.4f}".format(row_diff))
    
    total_before, total_after = mat.sum(), mat_rounded.sum()
    total_diff = np.max(total_after - total_before)
    print("Before rounding = {:.4f}".format(total_before))
    print("After rounding  = {:.4f}".format(total_after))
    print("Total difference = {:.4f}".format(total_diff))
    return mat_rounded

In [77]:
mat1_rounded = test_bucket_rounding(3, input_matrix=mat1)
mat1_rounded

Test Matrix Shape = 3 * 3
Execution completed in 0.58 seconds
Maximum row difference = 0.4000
Before rounding = 5.2000
After rounding  = 5.0000
Total difference = -0.2000


array([[0, 1, 1],
       [1, 0, 0],
       [2, 0, 0]])

In [78]:
mat2_rounded = test_bucket_rounding(100)

Test Matrix Shape = 100 * 100
Execution completed in 0.00 seconds
Maximum row difference = 8.4641
Before rounding = 5000.2931
After rounding  = 5000.0000
Total difference = -0.2931


In [79]:
mat3_rounded = test_bucket_rounding(1000)

Test Matrix Shape = 1000 * 1000
Execution completed in 0.03 seconds
Maximum row difference = 47.8521
Before rounding = 500012.7307
After rounding  = 500013.0000
Total difference = 0.2693


In [80]:
mat4_rounded = test_bucket_rounding(5263)

Test Matrix Shape = 5263 * 5263
Execution completed in 0.63 seconds
Maximum row difference = 114.5581
Before rounding = 13849531.9681
After rounding  = 13849532.0000
Total difference = 0.0319
